<a href="https://colab.research.google.com/github/wandb/wandb-workspaces/blob/example-notebook/Workspace_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Sample Notebook for [Weights & Biases Programmatic Workspace](https://docs.wandb.ai/guides/app/pages/workspaces#saved-workspace-views)


### Overview
* [wandb-workspaces](https://github.com/wandb/wandb-workspaces/tree/main) is a Python library for programmatically working with Weights & Biases workspaces and Reports. You can define a workspace programmatically by defining the workspace's properties, such as its name, associated entity and project, and the sections it should contain.

* Programmatically create and customize workspaces by defining configurations, setting panel layouts, and organizing sections. Load and modify workspaces via URL, filter and group runs using expressions, and customize run appearances. Easily copy views across workspaces for integration.

### Known limitations
* Only compatible with Saved Views (URLs ending with ?nw=...). Clone a `Personal Workspace` as `Saved Views` for programmatic interaction.

### Notebook Instructions:

* Run each cell step by step. The notebook will log sample data to a project named `workspace-api-example`.
* The notebook will create the project, log sample a to it, and then demonstrate the workspace examples using this data.

# Install and Import Dependencies

In [ ]:
# Install dependencies
!pip install wandb wandb-workspaces

In [ ]:
# makes formatting nicer (you might want to use dark mode on colab)
%load_ext rich

In [ ]:
# Import dependencies

import os
import wandb
import wandb_workspaces.workspaces as ws
import wandb_workspaces.reports.v2 as wr # We use the Reports API panels

# Create a Project to interact with programmatically for notebook examples

In [ ]:
# Initialize Weights & Biases and Login
wandb.login()

# Function to create a new project and log sample data
def create_project_and_log_data():
    project = "workspace-api-example"  # Default project name

    # Initialize a run to log some sample data
    with wandb.init(project=project, name="sample_run") as run:
        for step in range(100):
            wandb.log({
                "Step": step,
                "val_loss": 1.0 / (step + 1),
                "val_accuracy": step / 100.0,
                "train_loss": 1.0 / (step + 2),
                "train_accuracy": step / 110.0,
                "f1_score": step / 100.0,
                "recall": step / 120.0,
            })
    return project

# Create a new project and log data
project = create_project_and_log_data()
entity = wandb.Api().default_entity

# Summary of Programmatic Workspace features:
The Weights & Biases Workspace API allows you to create, manage, and customize workspaces programmatically. These capabilities are designed to help you organize and visualize your machine learning experiments more effectively.


### 1. Workspace Creation
* **Define Create Workspaces:** You can create a workspace with a specific
name, associate it with a project, and organize it into sections.
* **Customization:** Customize each workspace with settings such as axis configurations, smoothing types, outlier handling, and more.

### 2. Sections and Panels
* **Sections:** Organize your workspace into multiple sections, each containing a specific set of panels. Sections can be opened or closed based on your preference.
* **Panels:** Add various types of panels such as line plots, bar plots, and scalar charts to visualize different metrics.
* **Layouts:** Customize the layout of panels within each section using standard or custom grid layouts.

### 3. Workspace Settings
* **Global Settings:** Configure global settings for the workspace, including axis settings, smoothing options, outlier handling, and tooltip settings.
* **Section and Panel Settings:** Override global settings at the section or panel level to provide more granular control over the visualization.

### 4. Run Management
* **Run Sets:** Manage sets of runs in the workspace. Apply filters to include or exclude specific runs based on metrics or metadata.
* **Grouping and Ordering:** Group and order runs within the run set to better analyze and compare different experiments.
* **Run Customization:** Customize individual run settings such as color and visibility within the workspace..

### 5. Filters and Queries
* **Filter Expressions:** Apply complex filter expressions to include or exclude runs based on specific criteria.
* **Regex Queries:** Use regular expressions to search and filter runs within the workspace.

### 6. Saving and Loading Workspaces
* **Save Workspaces:** Save the current state of a workspace, making it accessible as a `Saved view` from the Project.
* **Load Workspaces:** Load an existing workspace from a URL to review or modify it.
* **Clone Workspaces:** Save a modified workspace as a new view, allowing you to create multiple versions of a workspace without affecting the original.

In [ ]:
# See all available settings for workspaces, sections, and panels.
all_settings_objects = [x for x in dir(ws) if isinstance(getattr(ws, x), type)]
all_settings_objects

# Creating a `Saved View` with new sections and panels

* **Purpose:** Demonstrates how to create a new workspace and populate it with sections and panels.
* **Benefit:** Helps users organize and visualize their project metrics in a structured manner.



In [ ]:

def example_00(entity, project):
    workspace = ws.Workspace(
        name="Example W&B Workspace",
        entity=entity,
        project=project,
        sections=[
            ws.Section(
                name="Validation Metrics",
                panels=[
                    wr.LinePlot(x="Step", y=["val_loss"]),
                    wr.BarPlot(metrics=["val_accuracy"]),
                    wr.ScalarChart(metric="f1_score", groupby_aggfunc="mean"),
                ],
                is_open=True,
            ),
        ],
    )
    workspace.save()
    print("Sample Workspace saved. Click the link above to open the new workspace.")
    return workspace.url

workspace_url = example_00(entity, project)

# Loading and Cloning
* **Purpose:** Demonstrates how to load an existing workspace, modify it, and save it as a new view.
* **Benefit:** Allows users to duplicate and customize workspaces without affecting the original setup.

In [ ]:
def example_01(url):
    workspace = ws.Workspace.from_url(url)
    workspace.name = "Updated Workspace Name"
    workspace.save()

    print(f"Workspace saved as new view.")

example_01(workspace_url)

# Basic Settings

* **Purpose:** Demonstrates how to create a workspace, add sections with panels, and configure settings for the workspace and individual panels.
* **Benefit:** Provides users with the ability to customize their workspace layout and visualization settings.


In [ ]:
def example_02(entity, project):
    workspace = ws.Workspace(name="An example workspace", entity=entity, project=project)
    workspace.sections = [
        ws.Section(
            name="Validation",
            panels=[
                wr.LinePlot(x="Step", y=["val_loss"]),
                wr.LinePlot(x="Step", y=["val_accuracy"]),
                wr.ScalarChart(metric="f1_score", groupby_aggfunc="mean"),
                wr.ScalarChart(metric="recall", groupby_aggfunc="mean"),
            ],
            is_open=True,
        ),
        ws.Section(
            name="Training",
            panels=[
                wr.LinePlot(x="Step", y=["train_loss"]),
                wr.LinePlot(x="Step", y=["train_accuracy"]),
            ],
        ),
    ]

    workspace.settings = ws.WorkspaceSettings(
        x_min=0,
        x_max=75,
        smoothing_type="gaussian",
        point_visualization_method="bucketing",
    )

    section = workspace.sections[0]
    section.panel_settings = ws.SectionPanelSettings(
        x_min=25,
        x_max=50,
        smoothing_type="none",
    )

    panel = section.panels[0]
    panel.title = "Validation Loss Custom Title"
    panel.title_x = "Custom x-axis title"

    workspace.save()
    print("Workspace with custom settings saved.")

example_02(entity, project)

# Using Filters

* **Purpose**: Demonstrates how to use runs filter within a workspace.
* **Benefit**: Provides powerful filtering capabilities to focus on run data that match specific patterns.

In [ ]:
def example_03(entity, project):
    # Get all runs in the project
    runs = wandb.Api().runs(f"{entity}/{project}")

    # Apply multiple filters: val_loss < 0.1, val_accuracy > 0.8, and run name matches regex pattern
    workspace = ws.Workspace(
        name="Advanced Filtered Workspace with Regex",
        entity=entity,
        project=project,
        sections=[
            ws.Section(
                name="Advanced Filtered Section",
                panels=[
                    wr.LinePlot(x="Step", y=["val_loss"]),
                    wr.LinePlot(x="Step", y=["val_accuracy"]),
                ],
                is_open=True,
            ),
        ],
        runset_settings=ws.RunsetSettings(
            filters=[
                (ws.Metric("val_loss") < 0.1),  # Filter runs by the 'val_loss' metric
                (ws.Metric("val_accuracy") > 0.8),  # Filter runs by the 'val_accuracy' metric
                (ws.Metric("ID").isin([run.id for run in wandb.Api().runs(f"{entity}/{project}")])),
            ],
            regex_query=True,
        )
    )

    # Add regex search to match run names starting with 's'
    workspace.runset_settings.query = "^s"
    workspace.runset_settings.regex_query = True

    workspace.save()
    print("Workspace with advanced filters and regex search saved.")

example_03(entity, project)

# Changing Runs Colors
* **Purpose:** Demonstrates how to change the colors of the runs in a workspace.
* **Benefit:** Helps users visually distinguish between different runs by assigning unique colors.



In [ ]:
def example_04(entity, project):
    # Get all runs in the project
    runs = wandb.Api().runs(f"{entity}/{project}")

    # Dynamically assign colors to the runs
    run_colors = ['purple', 'orange', 'teal', 'magenta']
    run_settings = {}
    for i, run in enumerate(runs):
        run_settings[run.id] = ws.RunSettings(color=run_colors[i % len(run_colors)])

    workspace = ws.Workspace(
        name="Run Colors Workspace",
        entity=entity,
        project=project,
        sections=[
            ws.Section(
                name="Run Colors Section",
                panels=[
                    wr.LinePlot(x="Step", y=["val_loss"]),
                    wr.LinePlot(x="Step", y=["val_accuracy"]),
                ],
                is_open=True,
            ),
        ],
        runset_settings=ws.RunsetSettings(
            run_settings=run_settings
        )
    )

    workspace.save()
    print("Workspace with run colors saved.")

example_04(entity, project)

# Grouping Runs

* **Purpose**: Demonstrates how to group runs by specific metrics.
* **Benefit**: Helps users organize and analyze runs by grouping them based on shared characteristics.

In [ ]:
def example_05(entity, project):
    workspace = ws.Workspace(
        name="Grouped Runs Workspace",
        entity=entity,
        project=project,
        sections=[
            ws.Section(
                name="Grouped Runs",
                panels=[
                    wr.LinePlot(x="Step", y=["val_loss"]),
                    wr.LinePlot(x="Step", y=["val_accuracy"]),
                ],
                is_open=True,
            ),
        ],
        runset_settings=ws.RunsetSettings(
            groupby=[ws.Metric("Name")]
        )
    )
    workspace.save()
    print("Workspace with grouped runs saved.")

example_05(entity, project)

# Sorting Runs

* **Purpose**: Demonstrates how to sort runs by specific metrics.
* **Benefit**: Allows users to easily identify and focus on the most relevant runs based on their sorting criteria.

In [ ]:
def example_06(entity, project):
    workspace = ws.Workspace(
        name="Sorted Runs Workspace",
        entity=entity,
        project=project,
        sections=[
            ws.Section(
                name="Sorted Runs",
                panels=[
                    wr.LinePlot(x="Step", y=["val_loss"]),
                    wr.LinePlot(x="Step", y=["val_accuracy"]),
                ],
                is_open=True,
            ),
        ],
        runset_settings=ws.RunsetSettings(
            order=[ws.Ordering(ws.Metric("val_loss"))]
        )
    )
    workspace.save()
    print("Workspace with sorted runs saved.")

example_06(entity, project)

# Putting it all together: creating and editing a custom Workspace

* **Purpose**: Demonstrates how to create a comprehensive workspace, configure its settings, and add panels to sections.
* **Benefit**: Provides a complete example of setting up and customizing a workspace programmatically.

In [ ]:
def example_08(entity, project):
    # Get all runs in the project
    runs = wandb.Api().runs(f"{entity}/{project}")

    # Dynamically assign colors to the runs and create run settings
    run_colors = ['red', 'blue', 'green', 'orange', 'purple', 'teal', 'magenta', '#FAC13C']
    run_settings = {}
    for i, run in enumerate(runs):
        run_settings[run.id] = ws.RunSettings(color=run_colors[i % len(run_colors)], disabled=False)

    workspace = ws.Workspace(
        name="My Workspace Template",
        entity=entity,
        project=project,
        sections=[
            ws.Section(
                name="Main Metrics",
                panels=[
                    wr.LinePlot(x="Step", y=["val_loss"]),
                    wr.LinePlot(x="Step", y=["val_accuracy"]),
                    wr.ScalarChart(metric="f1_score", groupby_aggfunc="mean"),
                ],
                is_open=True,
            ),
            ws.Section(
                name="Additional Metrics",
                panels=[
                    wr.ScalarChart(metric="precision", groupby_aggfunc="mean"),
                    wr.ScalarChart(metric="recall", groupby_aggfunc="mean"),
                ],
            ),
        ],
        settings=ws.WorkspaceSettings(
            x_axis="Step",
            x_min=0,
            x_max=100,
            smoothing_type="none",
            smoothing_weight=0,
            ignore_outliers=False,
            remove_legends_from_panels=False,
            tooltip_number_of_runs="default",
            tooltip_color_run_names=True,
            max_runs=20,
            point_visualization_method="bucketing",
            auto_expand_panel_search_results=False,
        ),
        runset_settings=ws.RunsetSettings(
            query="",
            regex_query=False,
            filters=[
                ws.Metric("val_loss") < 1,
                ws.Metric("Name") == "sample_run",
            ],
            groupby=[ws.Metric("Name")],
            order=[ws.Ordering(ws.Metric("Step"), ascending=True)],
            run_settings=run_settings
        )
    )
    workspace.save()
    print("Workspace created and saved.")

example_08(entity, project)